# Model Training Notebook

In [1]:
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset

from simulator.objects.policies.architectures.perceptron import MultiLayerPerceptron
from simulator.objects.policies.architectures import ModelTask
from simulator.objects.stock import Stock


torch.manual_seed(0)


In [2]:
def generate_example_stock_features(n_stocks: int) -> torch.Tensor:
    output = []
    noise_dist = torch.distributions.Normal(loc=0, scale=0.001)
    for _ in range(n_stocks):
        cash = torch.rand(size=(1,)) * 10000 - 2000
        earning_value_of_assets = torch.rand(size=(1,)) * 20000
        latest_quarterly_earnings = torch.rand(size=(1,)) * 20000
        start_price = torch.rand(size=(1,)) * 90 + 10
        price_slope = torch.rand(size=(1,)) * 0.01 - 0.005
        growth_component = start_price + price_slope * torch.arange(0, 1825)
        noise_component = noise_dist.sample(sample_shape=(1825,))
        price_history = growth_component + noise_component
        quality_of_leadership = torch.rand(size=(1,))
        stock = Stock(
            cash=cash.item(),
            earning_value_of_assets=earning_value_of_assets.item(),
            latest_quarterly_earnings=latest_quarterly_earnings.item(),
            price_history=price_history.numpy(),
            quality_of_leadership=quality_of_leadership.item(),
            stock_volatility=0.5
        )

        output.append(stock.get_stock_features())

    return torch.tensor(output)

class StockDataset(Dataset):
    def __init__(self, data, labels):
        self.data = torch.tensor(data, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"device: {device}")

device: cuda


In [4]:
N_SAMPLES = 5000
VALID_RATIO = 0.15
TEST_RATIO = 0.1

stock_features = generate_example_stock_features(N_SAMPLES)
stock_labels = stock_features[:, 0]

stock_dataset = StockDataset(stock_features, stock_labels)
val_length = int(N_SAMPLES * VALID_RATIO)
test_length = int(N_SAMPLES * TEST_RATIO)
train_length = N_SAMPLES - (val_length + test_length)

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(stock_dataset, lengths=[train_length, val_length, test_length])

train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=val_length, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=test_length, shuffle=True)

C:\Users\pomer\AppData\Local\Temp\ipykernel_30128\894559234.py:25: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:256.)
  return torch.tensor(output)
C:\Users\pomer\AppData\Local\Temp\ipykernel_30128\894559234.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.data = torch.tensor(data, dtype=torch.float32)
C:\Users\pomer\AppData\Local\Temp\ipykernel_30128\894559234.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.labels = torc

In [5]:
N_EPOCHS = 50

model = MultiLayerPerceptron(
    in_channels=13,
    hidden_channels=[16, 32],
    n_classes=1, 
    model_task=ModelTask.REGRESSOR
).to(device)

optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
loss_fun = torch.nn.MSELoss()

for i in range(N_EPOCHS):
    train_features, train_labels = next(iter(train_dataloader))
    val_features, val_labels = next(iter(val_dataloader))

    train_features = train_features.to(device)
    train_labels = train_labels.to(device)
    val_features = val_features.to(device)
    val_labels = val_labels.to(device)

    preds = model(train_features)
    loss = loss_fun(preds, train_labels)

    val_preds = model(val_features)
    val_loss = loss_fun(val_preds, val_labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f"Epoch: {i}; training_loss: {loss}; validation_loss: {val_loss}")



Epoch: 0; training_loss: 547140.875; validation_loss: 520693.59375
Epoch: 1; training_loss: 508854.1875; validation_loss: 458522.03125
Epoch: 2; training_loss: 450500.28125; validation_loss: 401043.15625
Epoch: 3; training_loss: 363854.8125; validation_loss: 348245.46875
Epoch: 4; training_loss: 312204.375; validation_loss: 300271.5
Epoch: 5; training_loss: 253293.890625; validation_loss: 256847.734375
Epoch: 6; training_loss: 214372.5; validation_loss: 218066.453125
Epoch: 7; training_loss: 155668.0; validation_loss: 183620.25
Epoch: 8; training_loss: 156050.46875; validation_loss: 153664.78125
Epoch: 9; training_loss: 144107.96875; validation_loss: 127444.140625
Epoch: 10; training_loss: 117594.421875; validation_loss: 104827.2109375
Epoch: 11; training_loss: 86032.75; validation_loss: 85707.6484375
Epoch: 12; training_loss: 70727.9921875; validation_loss: 69893.484375
Epoch: 13; training_loss: 55840.4453125; validation_loss: 57021.9609375
Epoch: 14; training_loss: 46016.546875; vali

In [7]:
test_features, test_labels = next(iter(test_dataloader))
print(loss_fun(model(test_features.to(device)), test_labels.to(device)))

tensor(7980.4883, device='cuda:0', grad_fn=<MseLossBackward0>)


In [ ]:
torch.save(model, "model.pt")